# Predicting Product Ratings From Web Browsing Data

In [1]:
import numpy as np
import pandas as pd
from matrix_completion import MatrixCompletionizationizer
from utils import *

# load the browsing history data
user_history = pd.read_csv("user_history.csv")
user_history_without_user_ID = user_history.drop(['USER ID'],axis=1)
user_history_indexed = user_history.set_index('USER ID')
user_ratings = pd.read_csv("user_ratings.csv")
user_history_indexed_normalized = user_history_indexed.copy()
user_history_indexed_normalized -= user_history_indexed_normalized.mean()
user_history_indexed_normalized /= user_history_indexed_normalized.std()

# load the ratings data
train, test = split_data('user_ratings.csv')

In [2]:
FEATURE_SELECTION_METHODS = ["All features", "PCA", "KMeans then PCA", "all"]
MATRIX_COMPLETION_METHODS = ["Use web features as initial user features", "Append web features"]

FEATURE_SELECTION = FEATURE_SELECTION_METHODS[2] #set this to 0, 1, or 2, or 3 for "all"
MATRIX_COMPLETION = MATRIX_COMPLETION_METHODS[1] #set this to 0 or 1

#set hyperparameters here:
learning_rate = 0.005
num_epochs = 2000
batch_size = 3000
k = 15

## I. Using unmodified web browsing data as features

In [3]:
if FEATURE_SELECTION == "All features" or FEATURE_SELECTION == "all":
    row_id_to_ind = dict(zip(user_history['USER ID'].to_numpy(), range(len(user_history['USER ID'].to_numpy()))))
    n_rows = len(row_id_to_ind)
    col_id_to_ind = dict(zip(train['PRODUCT'].unique(), range(len(train['PRODUCT'].unique()))))
    n_cols = len(col_id_to_ind)
    
    reduced_features = user_history_indexed_normalized

    added_cols = reduced_features
    added_col_id_to_ind = dict(zip(added_cols.columns.to_numpy(), range(n_cols, n_cols+added_cols.shape[1])))
    all_cols_id_to_ind = {**col_id_to_ind, **added_col_id_to_ind}
    added_cols_melted = added_cols.reset_index(inplace=False).melt('USER ID', var_name='PRODUCT', value_name='RATING')
    n_all_cols = len(all_cols_id_to_ind)

    train_appended = pd.concat([train, added_cols_melted], axis=0) 
    train_appended = train_appended.reset_index().drop(columns=['index'])

    train_ratings = train['RATING']
    train_rows = train['USER ID']
    train_cols = train['PRODUCT']

    train_ratings_appended = train_appended['RATING']
    train_rows_appended = train_appended['USER ID']
    train_cols_appended = train_appended['PRODUCT']

    test_ratings = test['RATING']
    test_rows = test['USER ID']
    test_cols = test['PRODUCT']

    row_features = reduced_features.to_numpy()
    
    if MATRIX_COMPLETION == "Append web features":
        model = MatrixCompletionizationizer(n_rows, n_all_cols, k=k)
        predicted = model.train(train_rows_appended, train_cols_appended, train_ratings_appended,
                                row_id_to_ind, all_cols_id_to_ind,
                                num_epochs=num_epochs, batch_size=batch_size, learning_rate = learning_rate,
                                print_loss = True, graph_loss = True, 
                                original_train_rows=train_rows, original_train_cols=train_cols, 
                                original_train_vals=train_ratings, original_col_id_to_ind=col_id_to_ind)
    elif MATRIX_COMPLETION == "Use web features as initial user features":
        model = MatrixCompletionizationizer(n_rows, n_cols, row_features=row_features)        
        predicted = model.train(train_rows, train_cols, train_ratings,
                                row_id_to_ind, col_id_to_ind,
                                num_epochs=num_epochs, batch_size=batch_size, learning_rate = learning_rate,
                                print_loss = True, graph_loss = True)

    test_rows_inds = np.array([row_id_to_ind[r] for r in test_rows])
    test_cols_inds = np.array([col_id_to_ind[c] for c in test_cols])

    print()
    print("Test MSE for using unprocessed history matrix:")
    print(mse(test_ratings, predicted[test_rows_inds, test_cols_inds]))
    print()
    print("Average test error (square root of MSE) for using unprocessed history matrix:")
    print(np.sqrt(mse(test_ratings, predicted[test_rows_inds, test_cols_inds])))
    print()

## II. Using web browsing data after PCA

In [4]:
if FEATURE_SELECTION == "PCA" or FEATURE_SELECTION == "all":
    row_id_to_ind = dict(zip(user_history['USER ID'].to_numpy(), range(len(user_history['USER ID'].to_numpy()))))
    n_rows = len(row_id_to_ind)
    col_id_to_ind = dict(zip(train['PRODUCT'].unique(), range(len(train['PRODUCT'].unique()))))
    n_cols = len(col_id_to_ind)

    reduced_features = PCA(user_history_indexed_normalized, user_history['USER ID'],
                           spectrum_fraction_threshold=0.9, graph_spectrum_fractions=False)

    added_cols = reduced_features
    added_col_id_to_ind = dict(zip(added_cols.columns.to_numpy(), range(n_cols, n_cols+added_cols.shape[1])))
    all_cols_id_to_ind = {**col_id_to_ind, **added_col_id_to_ind}
    added_cols_melted = added_cols.reset_index(inplace=False).melt('USER ID', var_name='PRODUCT', value_name='RATING')
    n_all_cols = len(all_cols_id_to_ind)

    train_appended = pd.concat([train, added_cols_melted], axis=0) 
    train_appended = train_appended.reset_index().drop(columns=['index'])

    train_ratings = train['RATING']
    train_rows = train['USER ID']
    train_cols = train['PRODUCT']

    train_ratings_appended = train_appended['RATING']
    train_rows_appended = train_appended['USER ID']
    train_cols_appended = train_appended['PRODUCT']

    test_ratings = test['RATING']
    test_rows = test['USER ID']
    test_cols = test['PRODUCT']

    row_features = reduced_features.to_numpy()
    
    if MATRIX_COMPLETION == "Append web features":
        model = MatrixCompletionizationizer(n_rows, n_all_cols, k=k)
        predicted = model.train(train_rows_appended, train_cols_appended, train_ratings_appended,
                                row_id_to_ind, all_cols_id_to_ind,
                                num_epochs=num_epochs, batch_size=batch_size, learning_rate = learning_rate,
                                print_loss = True, graph_loss = True, 
                                original_train_rows=train_rows, original_train_cols=train_cols, 
                                original_train_vals=train_ratings, original_col_id_to_ind=col_id_to_ind)
    elif MATRIX_COMPLETION == "Use web features as initial user features":
        model = MatrixCompletionizationizer(n_rows, n_cols, row_features=row_features)        
        predicted = model.train(train_rows, train_cols, train_ratings,
                                row_id_to_ind, col_id_to_ind,
                                num_epochs=num_epochs, batch_size=batch_size, learning_rate = learning_rate,
                                print_loss = True, graph_loss = True)

    test_rows_inds = np.array([row_id_to_ind[r] for r in test_rows])
    test_cols_inds = np.array([col_id_to_ind[c] for c in test_cols])

    print()
    print("Test MSE for PCA on entire history matrix:")
    print(mse(test_ratings, predicted[test_rows_inds, test_cols_inds]))
    print()
    print("Average test error (square root of MSE) for PCA on entire history matrix:")
    print(np.sqrt(mse(test_ratings, predicted[test_rows_inds, test_cols_inds])))
    print()

## III. Using web browsing data after PCA on individual clusters

In [ ]:
if FEATURE_SELECTION == "KMeans then PCA" or FEATURE_SELECTION == "all":
    num_clusters = 3
    reduced_cluster_data, user_history_clustered = KMeansPCA(user_history_indexed_normalized, user_history['USER ID'], 'USER ID', 
                                                             num_clusters=num_clusters, spectrum_fraction_threshold=0.9,
                                                             plot_clusters=False)
    train_with_clusters = train.merge(pd.DataFrame(user_history_clustered['Cluster']).reset_index())
    test_with_clusters = test.merge(pd.DataFrame(user_history_clustered['Cluster']).reset_index())

    for cluster_ind in range(num_clusters):

        train = train_with_clusters[train_with_clusters['Cluster']==cluster_ind].drop(columns=['Cluster'])
        test = test_with_clusters[test_with_clusters['Cluster']==cluster_ind].drop(columns=['Cluster'])

        cluster_inds = reduced_cluster_data[cluster_ind].reset_index()['USER ID'].unique()
        row_id_to_ind_cluster = dict(zip(cluster_inds, range(len(cluster_inds))))
        col_id_to_ind = dict(zip(train['PRODUCT'].unique(), range(len(train['PRODUCT'].unique()))))
        n_rows = len(row_id_to_ind_cluster)
        n_cols = len(col_id_to_ind)

        reduced_features = reduced_cluster_data[cluster_ind].sort_index()
        added_cols = reduced_features
        added_col_id_to_ind = dict(zip(added_cols.columns.to_numpy(), range(n_cols, n_cols+added_cols.shape[1])))
        all_cols_id_to_ind = {**col_id_to_ind, **added_col_id_to_ind}
        added_cols_melted = added_cols.reset_index(inplace=False).melt('USER ID', var_name='PRODUCT', value_name='RATING')
        n_all_cols = len(all_cols_id_to_ind)

        train_appended = pd.concat([train, added_cols_melted], axis=0) 
        train_appended = train_appended.reset_index().drop(columns=['index'])

        train_ratings = train['RATING']
        train_rows = train['USER ID']
        train_cols = train['PRODUCT']

        train_ratings_appended = train_appended['RATING']
        train_rows_appended = train_appended['USER ID']
        train_cols_appended = train_appended['PRODUCT']

        test_ratings = test['RATING']
        test_rows = test['USER ID']
        test_cols = test['PRODUCT']

        row_features = reduced_features.to_numpy()
    
        if MATRIX_COMPLETION == "Append web features":
            model = MatrixCompletionizationizer(n_rows, n_all_cols, k=k)
            predicted = model.train(train_rows_appended, train_cols_appended, train_ratings_appended,
                                    row_id_to_ind_cluster, all_cols_id_to_ind,
                                    num_epochs=num_epochs, batch_size=batch_size, learning_rate = learning_rate,
                                    print_loss = True, graph_loss = True, 
                                    original_train_rows=train_rows, original_train_cols=train_cols, 
                                    original_train_vals=train_ratings, original_col_id_to_ind=col_id_to_ind)
        elif MATRIX_COMPLETION == "Use web features as initial user features":
            model = MatrixCompletionizationizer(n_rows, n_cols, row_features=row_features)        
            predicted = model.train(train_rows, train_cols, train_ratings,
                                    row_id_to_ind_cluster, col_id_to_ind,
                                    num_epochs=num_epochs, batch_size=batch_size, learning_rate = learning_rate,
                                    print_loss = True, graph_loss = True)

        test_rows_inds = np.array([row_id_to_ind_cluster[r] for r in test_rows])
        test_cols_inds = np.array([all_cols_id_to_ind[c] for c in test_cols])
        print()
        print("Test MSE for cluster #" + str(cluster_ind) + ":")
        print(mse(test_ratings, predicted[test_rows_inds, test_cols_inds]))
        print()
        print("Average test error (square root of MSE) for cluster #" + str(cluster_ind) + ":")
        print(np.sqrt(mse(test_ratings, predicted[test_rows_inds, test_cols_inds])))
        print()

Train loss after epoch #1 is: 22.326847226082684
Train loss after epoch #200 is: 3.9231473576596305
Train loss after epoch #400 is: 0.8488907036923186
Train loss after epoch #600 is: 0.4079206399977904
Train loss after epoch #800 is: 0.2819348356540602
Train loss after epoch #1000 is: 0.22928117332218695
Train loss after epoch #1200 is: 0.19508237106721776
Train loss after epoch #1400 is: 0.17206247992473406
